In [12]:
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from avalanche.benchmarks.classic import SplitMNIST

from avalanche.evaluation.metrics import forgetting_metrics, \
accuracy_metrics, loss_metrics, timing_metrics, cpu_usage_metrics, \
confusion_matrix_metrics, disk_usage_metrics,bwt_metrics
from avalanche.models import SimpleMLP
from avalanche.logging import InteractiveLogger, TextLogger, TensorboardLogger, WandBLogger
from avalanche.training.plugins import EvaluationPlugin
from avalanche.training import Naive,EWC
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop, Resize


import wandb
import timm

import avalanche

In [6]:
from torchvision.datasets import MNIST
from avalanche.benchmarks.datasets.dataset_utils import default_dataset_location

mnist_train = MNIST(
            root = default_dataset_location("mnist"),
            train=True,
            download=True,
            transform=ToTensor(),
        )
mnist_test = MNIST(
            root = default_dataset_location("mnist"),
            train=False,
            download=True,
            transform=ToTensor(),
        )

9913344it [00:00, 52545179.32it/s]                                                                                                                                                                                 


Extracting /home/grbagwe/.avalanche/data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to /home/grbagwe/.avalanche/data/mnist/MNIST/raw


29696it [00:00, 32267889.01it/s]                                                                                                                                                                                   

Extracting /home/grbagwe/.avalanche/data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to /home/grbagwe/.avalanche/data/mnist/MNIST/raw


1649664it [00:00, 19590955.15it/s]                                                                                                                                                                                 

Extracting /home/grbagwe/.avalanche/data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to /home/grbagwe/.avalanche/data/mnist/MNIST/raw


5120it [00:00, 7708125.08it/s]                                                                                                                                                                                     
/home/grbagwe/miniconda3/envs/fedconlearn/lib/python3.8/site-packages/torchvision/datasets/mnist.py:479: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Extracting /home/grbagwe/.avalanche/data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to /home/grbagwe/.avalanche/data/mnist/MNIST/raw
Processing...
Done!


In [10]:
from avalanche.benchmarks import PermutedMNIST, nc_benchmark

benchmark = nc_benchmark(
            mnist_train, mnist_test, 5, task_labels=False, seed=1234
        )

In [31]:
a = next(iter(benchmark.train_stream))

TypeError: 'NCExperience' object is not iterable

In [25]:
benchmark.get_reproducibility_data()

{'class_ids_from_zero_from_first_exp': False,
 'class_ids_from_zero_in_each_exp': False,
 'class_mapping': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 'classes_order': [5, 4, 8, 2, 6, 9, 1, 7, 0, 3],
 'classes_order_original_ids': [5, 4, 8, 2, 6, 9, 1, 7, 0, 3],
 'n_classes_per_exp': [2, 2, 2, 2, 2],
 'n_experiences': 5,
 'has_task_labels': False}

In [27]:
benchmark.original_classes_in_exp

[{4, 5}, {2, 8}, {6, 9}, {1, 7}, {0, 3}]

In [7]:
model = SimpleMLP(hidden_size=256)

In [8]:
model

SimpleMLP(
  (features): Sequential(
    (0): Linear(in_features=784, out_features=256, bias=True)
    (1): ReLU(inplace=True)
    (2): Dropout(p=0.5, inplace=False)
  )
  (classifier): Linear(in_features=256, out_features=10, bias=True)
)

In [15]:
# loggers 
loggers = []

# log to Tensorboard
loggers.append(TensorboardLogger())

# log to text file
loggers.append(TextLogger(open('log.txt', 'a')))

# print to stdout
loggers.append(InteractiveLogger())

# W&B logger - comment this if you don't have a W&B account
loggers.append(WandBLogger(project_name="avalanche", run_name="ewc-SplitCifar-Vit"))

eval_plugin = EvaluationPlugin(
    accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
#     timing_metrics(epoch=True, epoch_running=True),
#     cpu_usage_metrics(experience=True),
    forgetting_metrics(experience=True, stream=True),
    bwt_metrics(experience=True, stream=True),
    confusion_matrix_metrics(num_classes=benchmark.n_classes, save_image=True,
                             stream=True),
#     disk_usage_metrics(minibatch=True, epoch=True, experience=True, stream=True),
    loggers=loggers
)


In [16]:
# CREATE THE STRATEGY INSTANCE (EWC)
cl_strategy = EWC(
    model,
    optimizer=SGD(model.parameters(), lr=0.001, momentum=0.9),
    criterion=CrossEntropyLoss(),
    ewc_lambda=0.4,
    train_mb_size=50, train_epochs=10,
    eval_mb_size=50,device='cuda',
    evaluator=eval_plugin)

In [17]:
# TRAINING LOOP
print('Starting experiment...')
results = []
for experience in benchmark.train_stream:
    print("Start training on experience ", experience.current_experience)

    cl_strategy.train(experience)
    print("End training on experience", experience.current_experience)
    print("Computing accuracy on the test set")
    results.append(cl_strategy.eval(benchmark.test_stream[:]))
        
        
# for experience in benchmark.train_stream:
#     # train returns a dictionary which contains all the metric values
#     res = cl_strategy.train(experience)
#     print('Training completed')

#     print('Computing accuracy on the whole test set')
    # test also returns a dictionary which contains all the metric values
    results.append(cl_strategy.eval(benchmark.test_stream))
    


Starting experiment...
Start training on experience  0
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 226/226 [00:03<00:00, 73.91it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.6992
	Loss_MB/train_phase/train_stream/Task000 = 0.1147
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8495
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 226/226 [00:02<00:00, 80.71it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1434
	Loss_MB/train_phase/train_stream/Task000 = 0.1058
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9623
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
100%|██████████████

-- Starting eval on experience 2 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 59.15it/s]
> Eval on experience 2 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp002 = 11.2796
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp002 = 0.0000
-- Starting eval on experience 3 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 44/44 [00:00<00:00, 61.99it/s]
> Eval on experience 3 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp003 = 7.8451
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.0000
-- Starting eval on experience 4 (Task 0) from test stream --
100%|████████████████████████████████████

	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp003 = 0.0000
-- Starting eval on experience 4 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:00<00:00, 60.88it/s]
> Eval on experience 4 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp004 = 10.3290
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp004 = 0.0000
-- >> End of eval phase << --
	ConfusionMatrix_Stream/eval_phase/test_stream = <avalanche.evaluation.metric_results.AlternativeValues object at 0x7fefacc84700>
	Loss_Stream/eval_phase/test_stream/Task000 = 7.2415
	StreamBWT/eval_phase/test_stream = -0.9904
	StreamForgetting/eval_phase/test_stream = 0.9904
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.1957
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|███████████████████████████

End training on experience 2
Computing accuracy on the test set
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 57.89it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	ExperienceBWT/eval_phase/test_stream/Task000/Exp000 = -0.9904
	ExperienceForgetting/eval_phase/test_stream/Task000/Exp000 = 0.9904
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 9.5804
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0000
-- Starting eval on experience 1 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 66.35it/s]
> Eval on experience 1 (Task 0) from test stream e

	StreamBWT/eval_phase/test_stream = -0.9870
	StreamForgetting/eval_phase/test_stream = 0.9870
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.2149
Start training on experience  4
-- >> Start of training phase << --
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [00:04<00:00, 59.11it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 1.0276
	Loss_MB/train_phase/train_stream/Task000 = 0.5945
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.7952
	Top1_Acc_MB/train_phase/train_stream/Task000 = 0.7500
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 242/242 [00:04<00:00, 60.20it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.1029
	Loss_MB/train_phase/train_stream/Task000 = 0.0276
	Top

In [18]:
import wandb
wandb.finish()

ExperienceBWT/eval_phase/test_stream/Task000/Exp000,▁▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp001,▁▁▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp002,█▁
ExperienceBWT/eval_phase/test_stream/Task000/Exp003,▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp000,▁▁▁▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp001,▁▁▁
ExperienceForgetting/eval_phase/test_stream/Task000/Exp002,▁█
ExperienceForgetting/eval_phase/test_stream/Task000/Exp003,▁
Loss_Epoch/train_phase/train_stream/Task000,▆▂▂▁▁▁▁▁█▂▂▂▂▂▂▁█▁▁▁▁▁▁▁█▁▁▁▁▁▁▁█▂▁▁▁▁▁▁
Loss_Exp/eval_phase/test_stream/Task000/Exp000,▁▆▇█▇
Loss_Exp/eval_phase/test_stream/Task000/Exp001,█▁▇██
